In [1]:
# Importar biliotecas necessárias
import pandas as pd
import numpy as np
import zipfile
import requests
from io import BytesIO
import os

In [7]:
# Criar um diretório para armazenar conteúdo do ENADE
os.makedirs('./gabriel_enade2019', exist_ok=True)

In [9]:
url = "https://download.inep.gov.br/microdados/microdados_enem_2019.zip"

# Faz o download do conteúdo
filebytes = BytesIO(
    requests.get(url).content
)

# Extrair o conteúdo do zipfile
myzip = zipfile.ZipFile(filebytes)
myzip.extractall("./gabriel_enade2019")

In [35]:
# Divide arquivo em mais partes por questões de memória filtrando por estado
chunk_size=1000000
batch_no=1
for chunk in pd.read_csv('./gabriel_enade2019/DADOS/MICRODADOS_ENEM_2019.csv', sep = ';', decimal = ',', chunksize=chunk_size, encoding = "ISO-8859-1"):
    chunk[chunk.SG_UF_RESIDENCIA == 'MG'].to_csv('./gabriel_enade2019/CHUNK/chunk'+str(batch_no)+'.csv', sep = ';', decimal = ',', encoding = "ISO-8859-1", index=False)
    batch_no+=1

In [36]:
# Cria um DataFrame único com todos os arquivos filtrados
columns= chunk.columns
enem = pd.DataFrame(columns = columns)
for i in range(1, batch_no):
    input = pd.read_csv('./gabriel_enade2019/CHUNK/chunk'+str(i)+'.csv', sep = ';', decimal = ',', encoding = "ISO-8859-1")
    enem = enem.append(input, ignore_index =True)

In [38]:
# Cria um arquivo CSV único com os dados filtrados 
enem.to_csv('./gabriel_enade2019/DADOS/MICRODADOS_ENEM_2019_MG.csv', sep = ';', decimal = ',', encoding = "ISO-8859-1", index=False)

In [46]:
enade = pd.read_csv(
    './gabriel_enade2019/DADOS/MICRODADOS_ENEM_2019_MG.csv',
    sep = ';', decimal = ',', encoding = "ISO-8859-1"
)

In [2]:

enem = pd.read_csv(
    "./gabriel_enade2019/DADOS/MICRODADOS_ENEM_2019.csv",
    sep = ";", decimal = ".", encoding = 'ISO-8859-1'
)

In [17]:
enem.loc[
    (enem.SG_UF_RESIDENCIA == "MG")&
 #   (enem.TP_SEXO == "M")&
 #   (enem.NO_MUNICIPIO_RESIDENCIA == "Sabará")&
    (enem.Q016 == "C")
    
].NU_NOTA_CH.describe()

count    2205.000000
mean      557.276599
std        90.127265
min         0.000000
25%       498.000000
50%       572.500000
75%       622.200000
max       775.500000
Name: NU_NOTA_CH, dtype: float64